# Amazon Review Data 활용하기
1. 데이터셋 버전
  - 2013 버전([링크](http://snap.stanford.edu/data/web-Amazon-links.html))
  - 2014 버전([링크](https://jmcauley.ucsd.edu/data/amazon/))
  - 2018 버전([링크](https://nijianmo.github.io/amazon/index.html))
2. 2018버전 데이터셋 정보
  - reviews(ratings, text, helpfulness votes)
  - product metadata(descriptions, category information, price band and image features)
  - links(also viewed/also bought graphs)
  - 이외 추가 정보는 웹사이트 참고
  - 최근(2020-08-07)까지 업데이트된 내용이 있다
3. 데이터셋 크기가 매우 크기 때문에, `Complete review data`보다 `Small subsets 데이터`를 먼저 사용하는 것을 추천한다

In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## data 다운로드하기

### 1. wget command 활용

In [2]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz

--2021-08-07 07:53:38--  http://deepyeti.ucsd.edu/jianmo/amazon/sample/meta_Computers.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7010521 (6.7M) [application/octet-stream]
Saving to: ‘meta_Computers.json.gz’

meta_Computers.json 100%[===================>]   6.69M  24.8MB/s    in 0.3s    

2021-08-07 07:53:38 (24.8 MB/s) - ‘meta_Computers.json.gz’ saved [7010521/7010521]



In [5]:
### load the meta data
data = []
with gzip.open('meta_Computers.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

18772
{'description': ['Brand new. Still in factory-sealed plastic.'], 'title': 'Kelby Training DVD: The Adobe Photoshop CS5 7-Point System for Camera Raw By Scott Kelby', 'image': ['https://images-na.ssl-images-amazon.com/images/I/31IlLImCVJL._SS40_.jpg'], 'brand': 'Kelby Training', 'rank': ['>#654,681 in Electronics (See Top 100 in Electronics)'], 'main_cat': 'Computers', 'date': 'December 2, 2011', 'asin': '0321732960'}


In [19]:
# convert list into pandas dataframe
df = pd.DataFrame.from_dict(data)
print(len(df))

18772


In [20]:
df.head()

,description,title,image,brand,rank,main_cat,date,asin,feature,tech1,also_buy,price,also_view,tech2,details,similar_item
0,[Brand new. Still in factory-sealed plastic.],Kelby Training DVD: The Adobe Photoshop CS5 7-...,[https://images-na.ssl-images-amazon.com/image...,Kelby Training,"[>#654,681 in Electronics (See Top 100 in Elec...",Computers,"December 2, 2011",0321732960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[If you're already a Photoshop user and want a...,Kelby Training DVD: Adobe Photoshop CS5 Power ...,[https://images-na.ssl-images-amazon.com/image...,Kelby Training,"[>#830,165 in Electronics (See Top 100 in Elec...",Computers,"November 7, 2011",0321735722,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[School To Home Notes: Note From the Teacher!\...,TEACHERS FRIEND NOTE FROM THE TEACHER SCHOOL T...,[https://images-na.ssl-images-amazon.com/image...,TEACHERS FRIEND,"[>#1,233,887 in Electronics (See Top 100 in El...",Computers,"June 6, 2007",0439651859,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,[Dooqi is a young brand specializing in high-e...,"[3-pack]Ipad Mini Screen Protector, Dooqi Scre...",[https://images-na.ssl-images-amazon.com/image...,dooqi,"[>#454,595 in Electronics (See Top 100 in Elec...",Computers,"December 5, 2014",0545729971,"[Hard coated for Scratch Proof (Hardness: 4H),...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[Cover for Nook HD],Hd+ Cheetah Cover- Nook,NaN,Nook,"[>#85,661 in Computers & Accessories (See top ...",Computers,"September 11, 2013",0594450284,[Cheetah cover],"{'Brand Name': 'Nook', 'Item Weight': '1 pound...",NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# remove rows with unformatted title (i.e. some 'title' may still contain html style content)
df_fillna = df.fillna('')
df_get_time = df_fillna[df_fillna.title.str.contains('getTime')] # unformatted rows
df_without_get_time = df_fillna[~df_fillna.title.str.contains('getTime')] # filter those unformatted rows
print(len(df_get_time))
print(len(df_without_get_time))

135
18637


In [26]:
# how those unformatted rows look like
df_get_time.iloc[0]

description                                                      
title           var aPageStart = (new Date()).getTime();\nvar ...
image                                                            
brand                                              DS Miller Inc.
rank            [>#1,826,312 in Electronics (See Top 100 in El...
main_cat                                                Computers
date                                                March 1, 2010
asin                                                   B0016C5EXY
feature                                                          
tech1                                                            
also_buy                                                         
price                                                            
also_view                                                        
tech2                                                            
details                                                          
similar_it

### 직접 다운로드

In [15]:
path = '/content/drive/MyDrive/data/amazon_reviews'

#### Read CSV file for ratings only

- (itme, user, rating, timestamp)

In [17]:
ratings_df = pd.read_csv(os.path.join(path, 'ratings_Musical_Instruments.csv'), header=None, names=['item','user','rating','timestamp'], encoding='utf-8')

In [18]:
print(ratings_df.shape)
print(ratings_df.head())

(500176, 4)
             item        user  rating   timestamp
0  A1YS9MDZP93857  0006428320     3.0  1394496000
1  A3TS466QBAWB9D  0014072149     5.0  1370476800
2  A3BUDYITWUSIS7  0041291905     5.0  1381708800
3  A19K10Z0D2NTZK  0041913574     5.0  1285200000
4  A14X336IB4JD89  0201891859     1.0  1350432000


In [27]:
data = []
with open(os.path.join(path, 'reviews_Musical_Instruments_5.json'), 'r', encoding='utf-8') as f:
    for l in f:
        data.append(json.loads(l.strip()))

In [28]:
all_df = pd.DataFrame.from_dict(data)
print(all_df.shape)
print(all_df.columns)

(10261, 9)
Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')


In [29]:
all_df = all_df[['reviewerID', 'asin', 'helpful', 'reviewText', 'overall']]
print(all_df.head())

       reviewerID  ... overall
0  A2IBPI20UZIR0U  ...     5.0
1  A14VAT5EAX3D9S  ...     5.0
2  A195EZSQDW3E21  ...     5.0
3  A2C00NNG1ZQQG2  ...     5.0
4   A94QU4C90B1AX  ...     5.0

[5 rows x 5 columns]


In [30]:
all_df.head()

,reviewerID,asin,helpful,reviewText,overall
0,A2IBPI20UZIR0U,1384719342,"[0, 0]","Not much to write about here, but it does exac...",5.0
1,A14VAT5EAX3D9S,1384719342,"[13, 14]",The product does exactly as it should and is q...,5.0
2,A195EZSQDW3E21,1384719342,"[1, 1]",The primary job of this device is to block the...,5.0
3,A2C00NNG1ZQQG2,1384719342,"[0, 0]",Nice windscreen protects my MXL mic and preven...,5.0
4,A94QU4C90B1AX,1384719342,"[0, 0]",This pop filter is great. It looks and perform...,5.0
